In [1]:
import os
import sys
#sys.path.append("/home/infomax/svc/lib")
import datetime
import requests
import logging
import time
import re
import lxml
from decimal import Decimal
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup

import pandas as pd

sURL = "https://www.youtube.com/watch?v=2wNTys-rPWs"

#https://www.youtube.com/watch?v=j45mjlHNnKs #토크멘터리
#https://www.youtube.com/watch?v=x6Y8S1msF14
#https://www.youtube.com/watch?v=Py5P4Ufv16Q
#https://www.youtube.com/watch?v=gwMa6gpoE9I #방탄 큰거

In [2]:
ops = webdriver.ChromeOptions()
#ops.add_argument('headless')
ops.add_argument('window-size=1920x860')
ops.add_argument('window-position=500,0')
#ops.add_argument("disable-gpu")
ops.add_argument(
"user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36")
ops.add_argument("lang=ko_KR")  # 한국어
ops.add_experimental_option("prefs", {
"profile.default_content_setting_values.automatic_downloads": True,
"directory_upgrade": True,
"safebrowsing.enabled": True})

In [3]:
with webdriver.Chrome(chrome_options=ops) as mydr:
    mydr.implicitly_wait(20)

#mydr.get("https://www.youtube.com/watch?v=Py5P4Ufv16Q")

    mydr.get(sURL)
    try:
        # 로딩까지 wait
        element_present = EC.presence_of_element_located((By.ID,'more'))
        WebDriverWait(mydr, 20).until(element_present)

    except TimeoutException:
        pass
    #oTitle = mydr.find_element_by_xpath('//h1[@class = "title style-scope ytd-video-primary-info-renderer"]')
    #oTitle.click()
    last_page_height = mydr.execute_script("return document.documentElement.scrollHeight")/3
    mydr.execute_script(f"window.scrollTo(0, {last_page_height});")

    print("wait for comments to load ...")
    WebDriverWait(mydr, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "ytd-comments ytd-comment-simplebox-renderer"))
    )
    oBtn_More = mydr.find_element_by_xpath('//paper-button[@id="more"]')
    oBtn_More.click()

    '''
    try:
        # 로딩까지 wait
        element_present = EC.presence_of_element_located((By.ID, 'sort-menu'))
        WebDriverWait(mydr, 5).until(element_present)
    except TimeoutException:
        pass
    '''
    last_size = 0
    nEndCount = 0
    nCount = 20
    for i in range(0,nCount):
        #new_page_height = mydr.execute_script("return document.documentElement.scrollHeight")/2
        lst_element = mydr.find_elements_by_xpath('//ytd-comment-renderer[@id="comment"]')
        last_element = lst_element[-1]
        mydr.execute_script("arguments[0].scrollIntoView(true);", last_element);
        time.sleep(0.5)
        new_size = len(lst_element)
        
        if new_size == last_size:
            nEndCount+= 1
            print(last_size)
            print(new_size)
        else:
            nEndCount = 0
        if nEndCount >1:
            print("break: "+ str(i))
            break
        last_size = new_size

    print('코멘트 클릭 시작')

    lst_more = mydr.find_elements_by_xpath('//paper-button[@id="more"]')
    for data in lst_more:
        if '자세히' in data.text:
            #mydr.execute_script("arguments[0].scrollIntoView(true);", data);
            mydr.execute_script("arguments[0].click();", data);
            time.sleep(1)
            #data.send_keys(Keys.Endter)

    print('코멘트 클릭 끝\n')

    sScripts = mydr.page_source

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: use options instead of chrome_options
  """Entry point for launching an IPython kernel.


wait for comments to load ...
40
40
60
60
80
80
100
100
120
120
140
140
160
160
180
180
코멘트 클릭 시작
코멘트 클릭 끝



In [4]:
parse = BeautifulSoup(sScripts,'lxml')

sYoutube_Title = parse.find('meta', {'property': 'og:title'})['content']
sYoutube_URL = parse.find('meta', {'property': 'og:url'})['content']
sYoutube_Image = parse.find('meta', {'property': 'og:image'})['content']

sYoutube_Count = parse.find('span',{'class':'view-count style-scope yt-view-count-renderer'}).text
rx = re.compile(r'([조회수 ,회])')
sYoutube_Count = rx.sub('',sYoutube_Count)

sYoutube_Date = parse.find('div',{'id':'date'}).find('yt-formatted-string', {'class':'style-scope ytd-video-primary-info-renderer'}).text
rx = re.compile(r'([ ])')
sYoutube_Date = rx.sub('',sYoutube_Date)

nUnit_Like = 1
nUnit_Dislike = 1
sYoutube_Like = parse.find('yt-formatted-string',{'aria-label':re.compile(r'좋아요')}).text
rx = re.compile(r'([천 만])')
if '천' in sYoutube_Like:
    sYoutube_Like = rx.sub('',sYoutube_Like)
    nUnit_Like = 1000
elif '만' in sYoutube_Like:
    sYoutube_Like = rx.sub('',sYoutube_Like)
    nUnit_Like = 10000
nYoutube_Like = int(float(sYoutube_Like) * nUnit_Like)


sYoutube_Dislike = parse.find('yt-formatted-string', {'aria-label': re.compile(r'싫어요')}).text
if '천' in sYoutube_Dislike:
    sYoutube_Dislike = rx.sub('',sYoutube_Dislike)
    nUnit_Dislike = 1000
elif '만' in sYoutube_Dislike:
    sYoutube_Dislike = rx.sub('',sYoutube_Dislike)
    nUnit_Dislike = 10000
nYoutube_Dislike = int(float(sYoutube_Dislike)*nUnit_Dislike)


#sYoutube_Discription = parse.find('meta', {'property': 'og:description'})['content']



print('제목: ' + sYoutube_Title)
print('URL: ' + sYoutube_URL)
print('Image_URL: ' + sYoutube_Image)
print('조회수: ' + sYoutube_Count)
print('게시일: ' + sYoutube_Date)
print('좋아요: ' + str(nYoutube_Like))
print('싫어요: ' + str(nYoutube_Dislike))


pdVideo_Info = pd.DataFrame({'sYoutube_Title':[sYoutube_Title],
                             'sYoutube_URL':[sYoutube_URL],
                             'sYoutube_Image':[sYoutube_Image],
                             'sYoutube_Count':[sYoutube_Count],
                             'sYoutube_Date':[sYoutube_Date],
                             'nYoutube_Like':[nYoutube_Like],
                             'nYoutube_Dislike':[nYoutube_Dislike]
                             })
pdVideo_Info

제목: (ENG SUB)제로콜라 외길 인생 40년!! 반백살의 절대미각 도전기 (코카콜라,펩X콜라,제로콜라) + 서핑보드 관리 꿀팁 대방출 | 와썹맨 | 미공개 하드털이 ep.2
URL: https://www.youtube.com/watch?v=2wNTys-rPWs
Image_URL: https://i.ytimg.com/vi/2wNTys-rPWs/maxresdefault.jpg
조회수: 556046
게시일: 2020.2.1.
좋아요: 8400
싫어요: 128


,sYoutube_Title,sYoutube_URL,sYoutube_Image,sYoutube_Count,sYoutube_Date,nYoutube_Like,nYoutube_Dislike
0,"(ENG SUB)제로콜라 외길 인생 40년!! 반백살의 절대미각 도전기 (코카콜라,...",https://www.youtube.com/watch?v=2wNTys-rPWs,https://i.ytimg.com/vi/2wNTys-rPWs/maxresdefau...,556046,2020.2.1.,8400,128


In [5]:
sYoutube_Channel_Nm = parse.find('ytd-channel-name',{'id':'channel-name'}).find('a').text

sYoutube_Channel_Sub_Count = parse.find('yt-formatted-string',{'id':'owner-sub-count'}).text
rx = re.compile(r'([구독자 만명])')
nUnit = 1
if '천' in sYoutube_Channel_Sub_Count:
    sYoutube_Channel_Sub_Count = rx.sub('', sYoutube_Channel_Sub_Count)
    nUnit = 1000
elif '만' in sYoutube_Channel_Sub_Count:
    sYoutube_Channel_Sub_Count = rx.sub('', sYoutube_Channel_Sub_Count)
    nUnit = 10000
nYoutube_Channel_Sub_Count = int(float(sYoutube_Channel_Sub_Count) * nUnit)

sYoutube_Discription = parse.find('div', {'id': 'description'}).text
sYoutube_Category = parse.find(text=re.compile(r'^카테고리')).find_parent().find_parent().find_next_sibling().find('a', {'class': 'yt-simple-endpoint style-scope yt-formatted-string'}).text
lst_Youtube_tag = parse.find_all('meta',{'property':'og:video:tag'})
sYoutube_tag = ', '.join(data['content'] for data in lst_Youtube_tag)

print('채널 명: ' + sYoutube_Channel_Nm)
print('구독자:' + str(nYoutube_Channel_Sub_Count))
print('------------------------------------------------')
print('Description: ' +sYoutube_Discription)
print('------------------------------------------------')
print('카테고리 :' + sYoutube_Category)     
print('------------------------------------------------')
print('tag: ' + sYoutube_tag)
print('------------------------------------------------')

pdChannel_Info = pd.DataFrame({'sYoutube_Channel_Nm': [sYoutube_Channel_Nm],
                               'nYoutube_Channel_Sub_Count': [nYoutube_Channel_Sub_Count],
                               'sYoutube_Description': [sYoutube_Discription],
                               'sYoutube_Category': [sYoutube_Category],
                               'sYoutube_tag': [sYoutube_tag]
                               })
pdChannel_Info

채널 명: 와썹맨-Wassup Man
구독자:2280000
------------------------------------------------
Description: 와썹맨의 새시즌을 기다리고이쓸 고마운 꼬맹쓰드를위해 두번째 서프라이즈를 준비해와썹!!
다들 설날은 잘 보내쮜? 벌써 2월이 되찌마는 와썹맨 열씨미 준비중이니까 쪽꿈만 더 기다려죠!!! BAAAAMM~~!!! 그리고! 나는 딱! 맛보면 다 안다고했짜너~!!!!
 

 
#와썹맨 #wassupman #박준형 #코카콜라 #콜라 #제로콜라 #콜라블라인드테스트 
 
[와썹맨] 1화 보러가기 ▶ http://bit.ly/2LvyvyV
[와썹맨] 2017년 ver. [사서고생_왓써맨] ▶ http://bit.ly/2AjbsSG
[스튜디오룰루랄라] 유튜브▶ https://www.youtube.com/studiolululala
------------------------------------------------
카테고리 :엔터테인먼트
------------------------------------------------
tag: 와썹맨, Wassupman, 박준형, 스튜디오룰루랄라, studiolululala, 리뷰, 비하인드, 미업로드, 미방분, 미업로드분, 코카콜라, 콜라, 블라인드테스트, 코믈리에, 콜라맛구별, 제로코크, 콜라블라인드, 절대미각, 미각테스트, 맛구별, 맛구분
------------------------------------------------


,sYoutube_Channel_Nm,nYoutube_Channel_Sub_Count,sYoutube_Description,sYoutube_Category,sYoutube_tag
0,와썹맨-Wassup Man,2280000,와썹맨의 새시즌을 기다리고이쓸 고마운 꼬맹쓰드를위해 두번째 서프라이즈를 준비해와썹!...,엔터테인먼트,"와썹맨, Wassupman, 박준형, 스튜디오룰루랄라, studiolululala,..."


In [6]:
pd_lstComments = pd.DataFrame({'sComments_Author': [],
                           'sComments_Time': [],
                           'sComments_text': [],
                           'nComments_Like': []
                           })
lst_comments = parse.find_all('ytd-comment-thread-renderer',{'class':'style-scope ytd-item-section-renderer'})
        
#print('======================================================')
for data in lst_comments:
    sAuthor = data.find('a',{'id':'author-text'}).text.strip()
    sTime = data.find('yt-formatted-string', {'class': re.compile('published-time-text')}).text.strip()
    sComment = data.find('yt-formatted-string',{'id':'content-text'}).text.replace('\n',' ').strip()
    sLike = data.find('span',{'id':'vote-count-middle'}).text.strip()
    rx = re.compile(r'([천 만])')
    nUnit_Like = 1
    if '천' in sLike:
        sLike = rx.sub('',sLike)
        nUnit_Like = 1000
    elif '만' in sLike:
        sLike = rx.sub('',sLike)
        nUnit_Like = 10000
    nLike = int(float(sLike) * nUnit_Like)
    #print(sAuthor)
    #print(sTime)
    #print(sComment)
    #print("좋아요: " + str(nLike))
    #print('======================================================')

    pdComments = pd.DataFrame({'sComments_Author': [sAuthor],
                           'sComments_Time': [sTime],
                           'sComments_text': [sComment],
                           'nComments_Like': [nLike]
                           })
    pd_lstComments = pd_lstComments.append(pdComments)
pd_lstComments.index = range(len(pd_lstComments))    
pd_lstComments
   

,sComments_Author,sComments_Time,sComments_text,nComments_Like
0,와썹맨-Wassup Man,3주 전(수정됨),저번에 올라간 와썹맨 Behind 꼬맹쓰들이 죠아해죠서 하나더 준비해봐썹!!! 벌써...,816.0
1,올마이티,3주 전,코카콜라 : 탄산이 더 강함 펩시: 단맛이 더 나고 탄산 덜함,131.0
2,라라루루,3주 전,이렇게 재밌는데 이제서야 공개하다니 제작진놈들이 일부러 숨긴게 분명하다!!!,665.0
3,채FILO,2주 전,누가 예전만 못하대ㅋㅋㅋ 그런 댓글 찾기도 힘들구만 괜히 지 댓글 더 띄우려고 소설...,157.0
4,Grumpy Firstborn,2주 전,How can someone be sexy while saying regular c...,9.0
5,comico19,3주 전,It really doesn't take a lot for Joon and his ...,33.0
6,/기울어,3주 전,Ah...... 그냥 복귀할 때까지 기다리면 되지 뭐 이렇게 생각했는데 오늘 오랜...,267.0
7,Jun C,3주 전,4:00 손가락으로 촉각 느끼신 이유는 제로는 설탕이 없기때문에 찐득함이 없는데 오...,284.0
8,su hyeon kim,3주 전,남들은 예전만 못하다지만 난 아직도 너무 재미있음 ㅋㅋㅋㅋ 시즌2 많이 기다리고 있...,252.0
9,VERY BLUE,3주 전,와 근데 이걸 진짜 어떻게 맞추지 4개는 진짜 헷갈릴텐디 역시 와썹맨👍👍👍 근데 겨...,356.0
